In [ ]:
%load_ext autoreload
%autoreload 2
%store -r the_editor

if 'the_editor' not in locals():
    import pickle
    print("Loading default data...")
    the_editor = pickle.load(open("data/the_editor.p",'rb'))

the_editor.to_frame('value')

from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# A. Actions per editor"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
from wikiwho_wrapper import WikiWho
import pandas as pd

wikiwho = WikiWho(lng='en')
agg_actions = wikiwho.dv.editions(editor_id = the_editor.userid)

# convert to datetime
agg_actions['year_month'] = pd.to_datetime(agg_actions['year_month'])

# define total columns
total_columns = ['total', 'total_surv_48h', 'total_persistent', 'total_stopword_count']

# add columns with the total actions
agg_actions = agg_actions.join(pd.DataFrame(
    agg_actions.loc[:,'adds':'adds_stopword_count'].values +\
    agg_actions.loc[:,'dels':'dels_stopword_count'].values +\
    agg_actions.loc[:,'reins':'reins_stopword_count'].values, 
    index=agg_actions.index, 
    columns=total_columns
))


import qgrid
qgrid.set_grid_option('maxVisibleRows', 5)
qgrid.show_grid(agg_actions[['year_month', 'page_id'] + total_columns])

In [ ]:
# Listener
from visualization.actions_listener import ActionsListener
listener = ActionsListener(agg_actions)

actions = (agg_actions.loc[:,'total':'total_stopword_count'].columns.append(
    agg_actions.loc[:,'adds':'reins_stopword_count'].columns)).values.tolist()


# Visualization
from utils.notebooks import get_date_slider_from_datetime
from ipywidgets import interact
interact(listener.listen, 
         _range = get_date_slider_from_datetime(agg_actions['year_month']),
         granularity=['Monthly', 'Yearly'],
         black=actions, 
         red=['total_surv_48h'] + actions + ['None'],
         green=['None'] + actions, 
         blue=['None'] + actions)


In [ ]:
from IPython.display import HTML
from utils.notebooks import get_next_notebook

display(HTML(f'<a href="{get_next_notebook()}" target="_blank">Go to next workbook</a>'))